In [1]:
import warnings
warnings.filterwarnings('ignore')

import segmentation_models_pytorch as smp
import torch
import os
import cv2
import timm
import albumentations as A
from skimage import io
import numpy as np
import segmentation_models_pytorch as smp
from skimage import io
from custom import U2NET

In [2]:
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'cspdarknet53',
 'cspdarknet53_iabn',
 'cspresnet50',
 'cspresnet50d',
 'cspresnet50w',
 'cspresnext50',
 'cspresnext50_iabn',
 'darknet53',
 'deit_base_distilled_patch16_224',
 'deit_base_distilled_patch16_384',
 'deit_base_patch16_224',
 'deit_base_patch16_384',
 'deit_small_distilled_patch16_224',
 'deit_small_patch16_224',
 'deit_tiny_distilled_patch16_224',
 'deit_tiny_patch16_224',
 'densenet121',
 'densenet121d',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenet264',
 'densenet264d_iabn',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2n

In [ ]:
transform = A.Compose([
    A.Resize(width=128, height=128),
    A.RandomRotate90(p = 1),
    A.Transpose(p=1),
    A.RandomBrightness(p=1),
    A.RandomContrast(p=1),
    A.RandomShadow(p=1),
    A.RGBShift(p=1),
    A.RandomContrast(p=1),
])

transform = A.Compose([
    A.Resize(width=128, height=128),
    A.GridDistortion(p=1),
    A.OpticalDistortion(p=1),
    A.ElasticTransform(p=1)

])

image =  io.imread(r"C:\Users\phili\Downloads\Telegram Desktop\CellsFinal\Cells\right side\cells\train\images\05__1_5291_9286.png")
mask = io.imread(r"C:\Users\phili\Downloads\Telegram Desktop\CellsFinal\Cells\right side\cells\train\labels\05__1_5291_9286.png", as_gray = True)
io.imsave("orig.png", image)
io.imsave("origmask.png", mask)
trans = transform(image = image, mask = mask)
image = trans["image"]
mask = trans["mask"]
io.imsave("image.png", image)
io.imsave("mask.png", mask)

In [ ]:
model = smp.Unet(encoder_name = "timm-resnest50d", classes = 2, in_channels = 3)
model.eval()

In [ ]:
model.load_state_dict(torch.load(r"E:\source\repos\VisualDL\timm-resnest50d, Unet.pt"))

In [ ]:
image_dir = r"C:\Users\phili\Downloads\Telegram Desktop\Cells\Cells\test\images"

In [ ]:
for cnt, im in enumerate(os.listdir(image_dir)):
    im = io.imread(os.path.join(image_dir, im))
    im = im/255.
    im = transform(image = im)["image"]
    im = torch.tensor(im, dtype = torch.float).permute(2, 0, 1).unsqueeze(0)
    print(im.shape)
    pred = model(im)
    pred = torch.argmax(pred, 1)[0] * 255.
    cv2.imwrite(str(cnt) + ".png", pred.numpy())

In [ ]:
ii = cv2.imread(r"C:\Users\phili\Downloads\Telegram Desktop\Cells1\Cells\train\labels\05__1_4685_10225.png",0)

In [ ]:
ii = ii * 255.

In [ ]:
cv2.imwrite("test.png", ii)

# Replace 2D with 3D

In [ ]:
from torch import nn

In [ ]:
for name, modules in model.named_modules():
    for module in modules:
        print(module)
    if(isinstance(module, nn.Conv2d)):
        kernel_size = module.kernel_size[0]
        stride = module.stride[0]
        padding = module.padding[0]
        weight = module.weight.unsqueeze(2) / kernel_size
        weight = torch.cat([weight for _ in range(0, kernel_size)], dim=2)
        bias = module.bias

        if(bias is None):
            print(modules)
            print(modules[name])
            modules[name] = nn.Conv3d(in_channels=module.weight.shape[1], out_channels=module.weight.shape[0],
                               kernel_size=kernel_size, padding=padding, stride=stride, bias=False)
        else:
            modules[name] = nn.Conv3d(in_channels=module.weight.shape[1], out_channels=module.weight.shape[0],
                               kernel_size=kernel_size, padding=padding, stride=stride, bias=True)
            modules[name].bias = bias

            modules[name].weight.data = weight

    elif(isinstance(module, nn.BatchNorm2d)):
        weight = module.weight
        bias = module.bias
        modules[name] = nn.BatchNorm3d(weight.shape[0])
        modules[name].weight = weight
        modules[name].bias = bias

for name in modules:
    parent_module = model
    objs = name.split(".")
    if len(objs) == 1:
        model.__setattr__(name, modules[name])
        continue

    for obj in objs[:-1]:
        parent_module = parent_module.__getattr__(obj)

    parent_module.__setattr__(objs[-1], modules[name])

In [ ]:
def get_all_modules(module):
    for a in module:
        get_all_modules(a.children())
        if isinstance(a, nn.Conv2d):
            print(a)
            #a = nn.Conv3d(3,32,3)
            
        elif isinstance(a, nn.BatchNorm2d):
            print(a)

            
  



In [ ]:
get_all_modules(model.children())

In [ ]:
for a in model.encoder.named_modules():
    print(a)
    print()

In [ ]:
import torch
from uformer_pytorch import Uformer

model = Uformer(
    dim = 16,           # initial dimensions after input projection, which increases by 2x each stage
    stages = 4,         # number of stages
    num_blocks = 2,     # number of transformer blocks per stage
    window_size = 16,   # set window size (along one side) for which to do the attention within
    dim_head = 64,
    heads = 1,
    ff_mult = 4
)
model.cuda()
x = torch.randn(1, 3, 512, 512).cuda()
pred = model(x) # (1, 3, 256, 256)

In [ ]:
pred.shape

# Segmentation inference

In [ ]:
from uformer_pytorch import Uformer
model = U2NET(3,2)

In [ ]:
model = smp.create_model(arch = "Unet", encoder_name = "resnext50_32x4d", classes = 2, in_channels = 3)
model.load_state_dict(torch.load(r"E:\source\repos\VisualDL\resnext50_32x4d, Unet.pt"))
model.eval()



In [ ]:
model = model.cuda()

In [ ]:
transform = A.Compose([
    A.Resize(width=128, height=128),
])
test = r"E:\source\repos\Daten\Cells\valid\images"
for cnt, im in enumerate(os.listdir(test)):
    image = io.imread(os.path.join(test, im))
    image = transform(image = image)["image"]
    image = image/255.
    s = torch.unsqueeze(torch.tensor(image, dtype = torch.float).permute(2, 0, 1), 0).cuda()
    with torch.cuda.amp.autocast():
        preds = model(s)
    preds = torch.argmax(preds, 1)
    preds *= 255
    #preds[preds == 1] = 50
    #preds[preds == 2] = 100
    #preds[preds == 3] = 150
    #preds[preds == 4] = 200
    #preds[preds == 5] = 250
    preds = preds.detach().cpu().numpy()
    io.imsave(f"{im}.png", preds[0])
    
    

# Transformer

In [ ]:
test = r"C:\Users\phili\Downloads\Telegram Desktop\Cells\train\labels"
ab = r"C:\Users\phili\Downloads\Telegram Desktop\Cells\train\bs"
for cnt, im in enumerate(os.listdir(test)):
    img = cv2.imread(os.path.join(test, im)) * 255.
    kernel = np.ones((3, 3), 'uint8')
    dilate_img = cv2.dilate(img, kernel, iterations=1)
    img1_bg = dilate_img - img
    img1 = img1_bg[:,:,0]
    clipped = np.clip(img1, 1, 6) # weight edges by factor (e.g. 6)
    print(np.min(clipped))
    cv2.imwrite(os.path.join(ab, im),clipped)

In [ ]:
from segmentation_models_pytorch.losses import DiceLoss

In [ ]:
DiceLoss(reduce = "none")

# Classification inference

In [ ]:
import timm
from skimage import io
import torch
import os
import numpy as np
import albumentations as A

In [ ]:
first = timm.create_model("resnext50_32x4d", pretrained=True, num_classes = 5).cuda()
#second = timm.create_model("resnext50d_32x4d", pretrained=True, num_classes = 5).cuda()
first.load_state_dict(torch.load(r"E:\source\repos\VisualDL\resnext50_32x4d.pt"))
#second.load_state_dict(torch.load(r"E:\source\repos\VisualDL\resnext50d_32x4d.pt"))
first.eval()
#second.eval()

In [ ]:
first = timm.create_model("resnext50_32x4d", pretrained=True, num_classes = 5).cuda()
second = timm.create_model("efficientnet_b4", pretrained=True, num_classes = 5).cuda()
first.load_state_dict(torch.load(r"E:\source\repos\VisualDL\resnext50_32x4d.pt"))
#second.load_state_dict(torch.load(r"E:\source\repos\VisualDL\efficientnet_b4.pt"))
first.eval()
#second.eval()

In [ ]:
path = ""
transform = A.Compose([
    A.Resize(width=512, height=512),
])
counter = 0
counterxd = 0
names = ["NA", "TRG0", "TRG1", "TRG2", "TRG3"]
for name in names:
    os.mkdir(name)
values = dict()
for cnt, name in enumerate(names):
    values[name] = []
    base = r"C:\Users\phili\Downloads\Telegram Desktop\dataset_valid/" + name
    for im in os.listdir(base):
        image = io.imread(os.path.join(base, im)).astype(np.float32)
        image = transform(image = image)["image"]
        image = image/255.
        s = torch.unsqueeze(torch.tensor(image, dtype = torch.float).permute(2, 0, 1), 0).cuda()
        preds =  first(s) 
        preds = torch.argmax(preds, 1)
        preds = preds.detach().cpu().numpy()
        values[name].append(preds[0])
        counterxd += 1
        io.imsave(f"{names[preds[0]]}/{im}.png", image)
        if cnt == preds:
            counter += 1
            
    

In [ ]:
counter/counterxd

In [ ]:
{'NA': [4, 0, 0, 2, 0, 0],
 'TRG0': [1, 4],
 'TRG1': [2, 2, 2],
 'TRG2': [3, 4, 3],
 'TRG3': [4, 4, 4, 4, 4, 4, 4, 0, 4, 2, 4, 4, 4, 4, 4, 4, 2]}

In [ ]:
path = r"C:\Users\phili\Downloads\Telegram Desktop\dataset_valid\TRG2\E93 L X20_0_1183_3925.png"
base = r"C:\Users\phili\Downloads\Telegram Desktop\dataset_valid\TRG2"


image = io.imread(path)
image = image/255.
s = torch.unsqueeze(torch.tensor(image, dtype = torch.float).permute(2, 0, 1), 0).cuda()
preds = first(s) + second(s)
preds = torch.argmax(preds, 1)
preds = preds.detach().cpu().numpy()
print(preds)
    

# Convert

In [ ]:
import os
import cv2
start = r"C:\Users\phili\Downloads\Telegram Desktop\test_data\test_data"

for im in os.listdir(os.path.join(start, "images")):
    img = cv2.imread(os.path.join(os.path.join(start, "images"), im), 0)
    mask = cv2.imread(os.path.join(os.path.join(start, "masks"), im), 0)
    mask[img == 0] = 0
    mask[mask == 0] = 0
    mask[mask == 29] = 1
    mask[mask == 105] = 2
    mask[mask == 117] = 3
    mask[mask ==189] = 4
    mask[mask == 225] = 5
    mask[mask > 5] = 1
    print(os.path.join(os.path.join(start, "labels"), im))
    cv2.imwrite(os.path.join(os.path.join(start, "labels"), im), mask)
    #img[img > 0] = 1
    #cv2.imwrite(os.path.join(start, im), img)

# Trian test split

In [ ]:
start = r"E:\source\repos\Daten\PLA\train\images"
to = r"E:\source\repos\Daten\PLA\val\images"
import random
from shutil import copyfile
import os
split = os.listdir(start)
random.shuffle(split)

test = split[0:35]
train = split[35:]

In [ ]:
for file in test:
    copyfile(os.path.join(start, file), os.path.join(to, file))
    copyfile(os.path.join(start, file).replace("images", "labels"), os.path.join(to, file).replace("images", "labels"))
    os.remove(os.path.join(start, file))
    os.remove(os.path.join(start, file).replace("images", "labels"))

In [ ]:
start = r"C:\Users\phili\Downloads\Telegram Desktop\Nuclei\Nuclei\train\labels"
out = r"C:\Users\phili\Downloads\Telegram Desktop\Nuclei\Nuclei\test"

In [ ]:
for file in os.listdir(start):
    preds = cv2.imread(os.path.join(start, file), 0)
    preds[preds == 1] = 50
    preds[preds == 2] = 100
    preds[preds == 3] = 150
    preds[preds == 4] = 200
    preds[preds == 5] = 250
    cv2.imwrite(os.path.join(out, file), preds)

# 255 to 1

In [ ]:
start = r"E:\source\repos\Daten\HER-N\hubt\dataset\Cells\valid\labels"
import cv2
import os
for file in os.listdir(start):
    img = cv2.imread(os.path.join(start, file), 0)
    img[img > 0] = 1
    cv2.imwrite(os.path.join(start, file), img)

# Transformer

In [ ]:
import torch
from uformer_pytorch import Uformer

model = model = Uformer(
                dim = 64,           # initial dimensions after input projection, which increases by 2x each stage
                stages = 3,         # number of stages
                num_blocks = 2,     # number of transformer blocks per stage
                window_size = 16,   # set window size (along one side) for which to do the attention within
                dim_head = 64,
                heads = 4,
                ff_mult = 2
            ).cuda()

x = torch.randn(2, 3, 128, 128).cuda()
with torch.cuda.amp.autocast():
    pred = model(x) # (1, 3, 256, 256)

# Extract single instances for obj. detec + semant. seg -> bring into yolov5 format

In [ ]:
import os
import cv2 as cv
import numpy as np

In [ ]:
im = cv.imread(r"E:\source\repos\Daten\Cells\train\labels\05__1_3115_9058.png", 0)

# Creating kernel
kernel = np.ones((2, 2), np.uint8)
  
# Using cv2.erode() method 
image = cv.erode(im, kernel) 

#image[image > 0] = 255
#ret, thresh = cv.threshold(im, 127, 255, 0)
contours,hierachy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

In [ ]:
len(contours)

In [ ]:
cv.imwrite("xd.png", image)

In [ ]:
files = {}
start = r"E:\source\repos\Daten\HER-N\hubt\dataset\Cells\train\labels"
all_files = os.listdir(start)
for img in os.listdir(start):
    print(img)
    im = cv.imread(os.path.join(start, img), 0)
    kernel = np.ones((2, 2), np.uint8)
    image = cv.erode(im, kernel) 
    contours,hierachy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    blank = np.zeros_like(im)
    for cnt, cont in enumerate(contours):
        xmin,ymin,width,height = cv.boundingRect(cont)
        print(width)
        #cv.rectangle(im,(x,y),(x+width,y+height),(255),1)
        #cv2.imwrite("xd.png", im)
        image_width = 256
        xcenter, ycenter = xmin + width/2, ymin + height/2
        xcenter, ycenter, width, height = xcenter/image_width, ycenter/image_width, width/image_width, height/image_width
        if not img in files:
            files[img] = [("0",str(xcenter), str(ycenter), str(width), str(height))]
        else:

            files[img] += [("0",str(xcenter), str(ycenter), str(width), str(height))]
        

In [ ]:
files

In [ ]:


base = cv2.imread(os.path.join(r"E:\source\repos\Daten\Cells\train\labels", '05__1_3115_10030.png'))

for rec in files['05__1_3115_10030.png']:
    xcenter,ycenter,width,height = [int(float(xx) * 128) for xx in rec[1:]]
    x = int(xcenter - width/2)
    y = int(ycenter - height/2)
    cv.rectangle(base,(x,y),(x+width,y+height),(255,0,0),1)
cv2.imwrite("xd2.png", base)

In [ ]:
im = cv.imread(os.path.join(r"E:\source\repos\Daten\Cells\train\labels", '05__1_3115_10030.png'), 0)
kernel = np.ones((2, 2), np.uint8)
image = cv.erode(im, kernel) 
contours,hierachy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
blank = np.zeros_like(im)
for cnt, cont in enumerate(contours):
    x,y,width,height = cv.boundingRect(cont)
    cv.rectangle(im,(x,y),(x+width,y+height),(255),1)
cv2.imwrite("xd.png", im)

In [ ]:
no_anno = [item for item in all_files if item not in list(files.keys())]

In [ ]:
os.mkdir("labels")
for cnt, (key, val) in enumerate(files.items()):
    with open("labels/" + key.replace(".png", ".txt"), "w") as handle:
        for va in val:
            handle.write(" ".join(list(va))+ "\n") 
            
for name in no_anno:
    with open("labels/" + name.replace(".png", ".txt"), "w") as handle:
        handle.write("")

# Export for instance segmentation

In [ ]:
import os
import cv2 as cv
import numpy as np
import random

In [ ]:
os.mkdir("labels")
os.mkdir("images")

In [ ]:
def cut_rec(img, rec):
    x,y,w,h = rec
    return img[y:y+h, x:x+w]
def add_rec(orig, img, rec):
    x,y,w,h = rec
    #r,g,b = [random.randint(20, 255) for i in range(3)]
    img[img > 0] = random.randint(20,255)
    tmp = np.expand_dims(img.astype(np.uint8), axis=-1)
    #tmp[np.all(tmp == (255, 255, 255), axis=-1)] = (b,g,r)
    orig[y:y+h, x:x+w] += tmp
    return orig

In [ ]:


pad = A.Compose([
    A.PadIfNeeded(64,64, value = 0, border_mode = 0),
])

In [ ]:
files = {}
labels = r"E:\source\repos\Daten\Cells\train\labels"
images = r"E:\source\repos\Daten\Cells\train\images"
to_labels = r"E:\source\repos\VisualDL\custom_experiments\labels"
to_images = r"E:\source\repos\VisualDL\custom_experiments\images"
all_files = os.listdir(labels)
adding = 0
for img in os.listdir(labels):
    im = cv.imread(os.path.join(labels, img), 0)
    original = cv.imread(os.path.join(images, img))
    kernel = np.ones((2, 2), np.uint8)
    image = cv.erode(im, kernel) 
    contours,hierachy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    for cnt, cont in enumerate(contours):
        blank = np.zeros_like(im)
        xmin,ymin,width,height = cv.boundingRect(cont)
        xmin -= 5
        ymin -= 5
        width +=5
        height += 5
        xmin = min(0, xmin)
        ymin = min(0, ymin)
        cv.drawContours(blank, [cont], -1, 1, -1)
        final = cut_rec(blank, (xmin, ymin , width , height ))
        orig_final = cut_rec(original, (xmin, ymin , width , height ))
        trans = pad(image = orig_final, mask = final)
        final = trans["mask"]
        orig_final = trans["image"]
        cv.imwrite(os.path.join(to_labels, img.replace(".png", f"{cnt}.png")), final)
        cv.imwrite(os.path.join(to_images, img.replace(".png", f"{cnt}.png")), orig_final)
        #cv.rectangle(im,(x,y),(x+width,y+height),(255),1)
        
        #cv2.imwrite("xd.png", im)
    


# instance segmentation inference

In [ ]:
model = smp.UnetPlusPlus(encoder_name = "resnext50_32x4d", classes = 2, in_channels = 3)
model.load_state_dict(torch.load(r"E:\source\repos\VisualDL\resnext50_32x4d, UnetPlusPlus.pt"))
model.eval()
model=model.cuda()


## simple predict

In [ ]:
transform = A.Compose([
    #A.PadIfNeeded(64,64, value = 0, border_mode = 0),
    A.Resize(width=64, height=64),
])
test = r"E:\source\repos\VisualDL\custom_experiments\valid\images"
for cnt, im in enumerate(os.listdir(test)):
    image = io.imread(os.path.join(test, im))
    image = transform(image = image)["image"]
    image = image/255.
    s = torch.unsqueeze(torch.tensor(image, dtype = torch.float).permute(2, 0, 1), 0).cuda()
    with torch.cuda.amp.autocast():
        preds = model(s)
    preds = torch.argmax(preds, 1)
    preds *= 255
    preds = preds.detach().cpu().numpy()
    io.imsave(f"{im}.png", preds[0])
    

# use rectangle

In [ ]:
transform = A.Compose([
    A.PadIfNeeded(64,64, value = 0, border_mode = 0),
    A.Resize(width=64, height=64),
])

files = {}
labels = r"E:\source\repos\Daten\Cells\valid\labels"
images = r"E:\source\repos\Daten\Cells\valid\images"
to = r"E:\source\repos\VisualDL\custom_experiments\test"
all_files = os.listdir(labels)
adding = 0
for ii, img in enumerate(os.listdir(labels)):
    im = cv.imread(os.path.join(labels, img), 0)
    print(img)
    original = cv.imread(os.path.join(images, img))
    blank = np.zeros_like(original)
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    kernel = np.ones((2, 2), np.uint8)
    image = cv.erode(im, kernel) 
    contours,hierachy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    for cnt, cont in enumerate(contours):
        
        xmin,ymin,width,height = cv.boundingRect(cont)
        orig_final = cut_rec(original, (xmin, ymin , width , height ))
        tt = orig_final.copy()
        orig_final = transform(image = orig_final)["image"]
        image = orig_final/255.
        s = torch.unsqueeze(torch.tensor(image, dtype = torch.float).permute(2, 0, 1), 0).cuda()
        with torch.cuda.amp.autocast():
            preds = model(s)
        preds = torch.argmax(preds, 1)
        preds *= 255
        preds = preds.detach().cpu().numpy()[0]

        preds = cut_rec(preds, (32 - int(width/2), 32 - int(height/2), width, height))
        #contours,hierachy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        blank = add_rec(blank, preds, (xmin,ymin,width,height))
        cv2.imwrite(os.path.join(to, f"{img}{cnt}.png"), preds)
        cv2.imwrite(os.path.join(to, f"{img}{cnt}orig.png"), tt)
    cv2.imwrite(os.path.join(to, f"{img}{cnt}xdddd.png"), blank)
    if ii == 2:
        break

In [ ]:
from skimage.transform import resize

In [ ]:
skimage.skimage.transform

In [ ]:
model = U2NET(3, 2)

In [ ]:
dummy_in = torch.randn(1, 3, 256, 256)

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, einsum
import numpy as np
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from custom import ViT, AxialImageTransformer, AxialAttention
# helpers
from torch import nn
from torch.nn import functional
def exists(val):
    return val is not None

def default(val, d):
    return val if exists(val) else d

def divisible_by(val, divisor):
    return (val % divisor) == 0

def unfold_output_size(image_size, kernel_size, stride, padding):
    return int(((image_size - kernel_size + (2 * padding)) / stride) + 1)

# classes

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, mult = 4, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim * mult),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim * mult, dim)
        )

    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(
        self,
        *,
        dim,
        heads = 8,
        dim_head = 64,
        dropout = 0.
    ):
        super().__init__()
        inner_dim = heads * dim_head
        self.heads =  heads
        self.scale = dim_head ** -0.5

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        b, n, d, h = *x.shape, self.heads
        q, k, v = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h = h), (q, k, v))

        sim = einsum('b i d, b j d -> b i j', q, k) * self.scale
        attn = sim.softmax(dim = -1)

        out = einsum('b i j, b j d -> b i d', attn, v)
        out = rearrange(out, '(b h) n d -> b n (h d)', h = h)
        return self.to_out(out)

# main class

class TNT(nn.Module):
    def __init__(
        self,
        *,
        image_size,
        patch_dim,
        pixel_dim,
        patch_size,
        pixel_size,
        depth,
        heads = 8,
        dim_head = 64,
        ff_dropout = 0.,
        attn_dropout = 0.,
        channels = 3,
        unfold_args = None
    ):
        super().__init__()
        assert divisible_by(image_size, patch_size), 'image size must be divisible by patch size'
        assert divisible_by(patch_size, pixel_size), 'patch size must be divisible by pixel size for now'

        num_patch_tokens = (image_size // patch_size) ** 2

        self.image_size = image_size
        self.patch_size = patch_size
        self.patch_tokens = nn.Parameter(torch.randn(num_patch_tokens + 1, patch_dim))

        unfold_args = default(unfold_args, (pixel_size, pixel_size, 0))
        unfold_args = (*unfold_args, 0) if len(unfold_args) == 2 else unfold_args
        kernel_size, stride, padding = unfold_args

        pixel_width = unfold_output_size(patch_size, kernel_size, stride, padding)
        num_pixels = pixel_width ** 2

        self.to_pixel_tokens = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> (b h w) c p1 p2', p1 = patch_size, p2 = patch_size),
            nn.Unfold(kernel_size = kernel_size, stride = stride, padding = padding),
            Rearrange('... c n -> ... n c'),
            nn.Linear(channels * kernel_size ** 2, pixel_dim)
        )

        self.patch_pos_emb = nn.Parameter(torch.randn(num_patch_tokens + 1, patch_dim))
        self.pixel_pos_emb = nn.Parameter(torch.randn(num_pixels, pixel_dim))

        layers = nn.ModuleList([])
        for _ in range(depth):

            pixel_to_patch = nn.Sequential(
                nn.LayerNorm(pixel_dim),
                Rearrange('... n d -> ... (n d)'),
                nn.Linear(pixel_dim * num_pixels, patch_dim),
            )

            layers.append(nn.ModuleList([
                PreNorm(pixel_dim, Attention(dim = pixel_dim, heads = heads, dim_head = dim_head, dropout = attn_dropout)),
                PreNorm(pixel_dim, FeedForward(dim = pixel_dim, dropout = ff_dropout)),
                pixel_to_patch,
                PreNorm(patch_dim, Attention(dim = patch_dim, heads = heads, dim_head = dim_head, dropout = attn_dropout)),
                PreNorm(patch_dim, FeedForward(dim = patch_dim, dropout = ff_dropout)),
            ]))

        self.layers = layers



    def forward(self, x):
        b, _, h, w, patch_size, image_size = *x.shape, self.patch_size, self.image_size
        assert divisible_by(h, patch_size) and divisible_by(w, patch_size), f'height {h} and width {w} of input must be divisible by the patch size'

        num_patches_h = h // patch_size
        num_patches_w = w // patch_size
        n = num_patches_w * num_patches_h

        pixels = self.to_pixel_tokens(x)
        patches = repeat(self.patch_tokens[:(n + 1)], 'n d -> b n d', b = b)

        patches += rearrange(self.patch_pos_emb[:(n + 1)], 'n d -> () n d')
        pixels += rearrange(self.pixel_pos_emb, 'n d -> () n d')

        for pixel_attn, pixel_ff, pixel_to_patch_residual, patch_attn, patch_ff in self.layers:

            pixels = pixel_attn(pixels) + pixels
            pixels = pixel_ff(pixels) + pixels

            patches_residual = pixel_to_patch_residual(pixels)

            patches_residual = rearrange(patches_residual, '(b h w) d -> b (h w) d', h = num_patches_h, w = num_patches_w)
            patches_residual = F.pad(patches_residual, (0, 0, 1, 0), value = 0) # cls token gets residual of 0
            patches = patches + patches_residual

            patches = patch_attn(patches) + patches
            patches = patch_ff(patches) + patches
        hidden_states = patches[:,1:,:]
        B, n_patch, hidden = hidden_states.size()  # reshape from (B, n_patch, hidden) to (B, h, w, hidden)
        h, w = int(np.sqrt(n_patch)), int(np.sqrt(n_patch))
        x = hidden_states.permute(0, 2, 1)
        x = x.contiguous().view(B, hidden, h, w)
        return x
    
    
class Conv2dReLU(nn.Sequential):
    def __init__(
            self,
            in_channels,
            out_channels,
            kernel_size,
            padding=0,
            stride=1,
            use_batchnorm=True,
    ):

        if use_batchnorm == "inplace" and InPlaceABN is None:
            raise RuntimeError(
                "In order to use `use_batchnorm='inplace'` inplace_abn package must be installed. "
                + "To install see: https://github.com/mapillary/inplace_abn"
            )

        conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            bias=not (use_batchnorm),
        )
        relu = nn.ReLU(inplace=True)

        if use_batchnorm == "inplace":
            bn = InPlaceABN(out_channels, activation="leaky_relu", activation_param=0.0)
            relu = nn.Identity()

        elif use_batchnorm and use_batchnorm != "inplace":
            bn = nn.BatchNorm2d(out_channels)

        else:
            bn = nn.Identity()

        super(Conv2dReLU, self).__init__(conv, bn, relu)

In [ ]:


image_size = 128
init_dim = 16
conv = Conv2dReLU(3, init_dim, 1).cuda()
inst = AxialImageTransformer(dim = init_dim,depth = 2,axial_pos_emb_shape = (image_size,image_size)).cuda()

first = TNT(image_size = image_size, patch_size = 2, pixel_dim = 16, pixel_size = 1, patch_dim = init_dim * 2, depth = 2, heads = 4, channels = init_dim).cuda()
second = TNT(image_size = image_size//2, patch_size = 2, pixel_dim = 16, pixel_size = 1, patch_dim = init_dim * 4, depth = 4, heads = 6, channels = init_dim * 2).cuda()
third = TNT(image_size = image_size//4, patch_size = 2, pixel_dim = 16, pixel_size = 1, patch_dim = init_dim * 8, depth = 6, heads = 8, channels = init_dim * 4).cuda()
fourth = TNT(image_size = image_size//8, patch_size = 2, pixel_dim = 16, pixel_size = 1, patch_dim = init_dim * 16, depth = 8, heads = 12, channels = init_dim * 8).cuda()
cup = ViT(
            image_size = 8,
            patch_size = 1,
            dim = init_dim * 32,
            depth = 6,
            heads = 12,
            mlp_dim = 2048,
            dropout = 0.1,
            emb_dropout = 0.1,
            channels = init_dim * 16
        ).cuda()

class DecoderBlock(nn.Module):
    def __init__(self,
                in_channels,
                skip_channels,
                out_channels,
                depth,
                size,
                heads= 2):
        super().__init__()   
        self.ax = AxialImageTransformer(dim = in_channels + skip_channels,heads= heads,depth = depth,axial_pos_emb_shape = (size,size)).cuda()
        self.out = Conv2dReLU(in_channels + skip_channels, out_channels, 1)
        
    def forward(self, x, skip = None):
        x = F.interpolate(x, scale_factor=2, mode="nearest")
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        return self.out(self.ax(x))
        

In [ ]:
nc = 2
up1 = DecoderBlock(init_dim*32, init_dim*8, init_dim*8, depth = 2, heads = 2,size = image_size//8).cuda()
up2 = DecoderBlock(init_dim*8, init_dim*4, init_dim*4, depth = 2, heads = 2,size = image_size//4).cuda()
up3 = DecoderBlock(init_dim*4, init_dim*2, init_dim*2, depth = 2, heads = 2,size = image_size//2).cuda()
up4 = DecoderBlock(init_dim*2, init_dim, 2, depth = 2, heads = 2,size = image_size).cuda()

In [ ]:
a=AxialAttention(32,dim_index =1).cuda()
a= AxialImageTransformer(dim = 3072, depth = 1,axial_pos_emb_shape = (128,128), heads=1).cuda()

In [ ]:
dummy_in = torch.randn(2, 3072, 128, 128).cuda()

In [ ]:
a(dummy_in).shape


In [ ]:
inst(conv(dummy_in)).shape

In [ ]:
def convert(hidden_states):
    B, n_patch, hidden = hidden_states.size()  # reshape from (B, n_patch, hidden) to (B, h, w, hidden)
    h, w = int(np.sqrt(n_patch)), int(np.sqrt(n_patch))
    x = hidden_states.permute(0, 2, 1)
    x = x.contiguous().view(B, hidden, h, w)
    return x

In [ ]:
with torch.cuda.amp.autocast():
    out1 = inst(conv(dummy_in)) #128
with torch.cuda.amp.autocast():
    out2 = first(out1) #64
with torch.cuda.amp.autocast():
    out3 = second(out2) #32
with torch.cuda.amp.autocast():
    out4 = third(out3) #16
with torch.cuda.amp.autocast():
    out5 = fourth(out4) #8
with torch.cuda.amp.autocast():
     out6 = cup(out5) #8



In [ ]:

up4(up3(up2(up1(out6, out4), out3), out2), out1).shape

In [ ]:

hidden_states = out
B, n_patch, hidden = hidden_states.size()  # reshape from (B, n_patch, hidden) to (B, h, w, hidden)
h, w = int(np.sqrt(n_patch)), int(np.sqrt(n_patch))
x = hidden_states.permute(0, 2, 1)
x = x.contiguous().view(B, hidden, h, w)

In [ ]:
x.shape

In [ ]:
2**5